In [1]:
import pandas as pd
import numpy as np
import glob
import os

import geopandas as gpd
import json
import requests
from shapely.geometry.linestring import LineString
from shapely.geometry.point import Point

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
from folium import plugins
import h3
import geopy.distance
from pyproj import Geod

import plotly
import plotly.express as px
import plotly.figure_factory as ff

from geojson import Feature, FeatureCollection
import geojson
import json
import matplotlib

# from lib.custom_functions import *

# Functions

In [2]:
def linestring_to_string(line):
    # Extract the coordinates from the LineString and format them
    coords = list(line.coords)
    coords_str = ','.join(f'({x},{y})' for x, y in coords)
    linestring_str = f'LineString([{coords_str}])'
    return linestring_str

In [3]:
def decode(encoded):
  inv = 1.0 / 1e6
  decoded = []
  previous = [0,0]
  i = 0
  # For each byte
  while i < len(encoded):
    # For each coord (lat, lon)
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      # Keep decoding bytes until you have this coord
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      # Get the final value adding the previous offset and remember it for the next
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    # Scale by the precision and chop off long coords also flip the positions so
    # Iits the far more standard lon,lat instead of lat,lon
    decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
  # Return the list of coordinates
  return decoded

In [4]:
def clutter_detection(data, w, r):
    '''
    data: a series containing 'lat' and 'lon' sorted in sequential order.
    r: the radius to identify the clutter. r can be estimate using the GPS error radius.

    return the index of the potential clutters
    '''
    core_list = data
    issue_list = []
    for i in range(1,len(core_list)):
        # print(i)
        start = max(0, i-int(w/2))
        stop = min(i+int(w/2), len(core_list)-1)
        if geopy.distance.geodesic(core_list[start], core_list[stop]).meters<r:
            issue_list.append(i)
    return issue_list

def clutter_grouping(data, w, s):
    '''
    data: a series of problematic points
    s: sensitivity determins how wide the gap is to merge two or more together into a bigger clutter
    w: the moving window size

    return a list of the start points and end points of the clutters
    '''
    if len(data)==0:
        return []

    clusters = []
    result = []
    current_group = [data[0]]

    for i in range(1, len(data)):
        # print(i)
        if data[i] - data[i-1] <= s:
            current_group.append(data[i])
        else:
            clusters.append(current_group)
            current_group = [data[i]]

    clusters.append(current_group)

    for j in clusters:
        # print(j)
        result.append([j[0]-int(w/2),j[-1]+int(w/2)])
    for k in range(0, len(result)-1):
        # print(k)
        if result[k][1]>result[k+1][0]:
            result[k][1] = int((result[k][1]+result[k+1][0])/2)
            result[k+1][0] = result[k][1]
    return result

def decluttering_data(data, w, r, s, lon_col_name='lon', lat_col_name='lat'):
    '''
    data: a Dataframe containing 'lat' and 'lon' sorted in sequential order.
    w: the moving window size
    r: the radius to identify the clutter. r can be estimate using the GPS error radius.
    s: sensitivity determins how wide the gap is to merge two or more together into a bigger clutter

    return the dataset clutters removed
    '''
    data = data.reset_index(drop=True)
    core_list = data[[lon_col_name,lat_col_name]].to_numpy()
    # print(core_list)
    issue_list = clutter_detection(core_list, w, r)
    # print(issue_list)
    clutter_list = clutter_grouping(issue_list, w, s)
    # print(clutter_list)

    if len(clutter_list) > 0:
        # A list of uncluttered indeces
        index_list = []
        index = 0
        for i in range(0, len(clutter_list)):
            if clutter_list[i][0]<=0:
                clutter_list[i][0]=0
            if clutter_list[i][1]>=len(core_list)-1:
                clutter_list[i][1]=len(core_list)-1
            while index<=clutter_list[i][0]:
                index_list.append(index)
                # print(index_list)
                index+=1
            index=clutter_list[i][1]
            index_list.append(index)
            index+=1
            if i==len(clutter_list)-1:
                while index<=len(core_list)-1:
                    index_list.append(index)  
                    index+=1  
            
        # print(index_list)
        # result_df = data.iloc[index_list]

        return data.iloc[index_list]
    else:
        return data

In [164]:
def map_matching(data, lon_col_name='Longitude', lat_col_name='Latitude', time_col_name='RecordedAtTime',
                 search_radius=5, costing='bus', turn_penalty_factor=100):
    '''
    data: a dataframe containing lon, lat and time (seconds)

    return two results:
        a linestring for estimated route
        a DataFrame contains matched data on the exact order of input data
    '''
    data = pd.DataFrame({
        'lon': data[lon_col_name],
        'lat': data[lat_col_name],
        'time': data[time_col_name].apply(lambda x: x.timestamp())
    }).reset_index(drop=True)

    meili_coordinates = data.to_json(orient='records')
    meili_head = '{"shape":'
    meili_tail = f""","search_radius": {search_radius}, "shape_match":"map_snap", "costing":"{costing}", "format":"osrm", "turn_penalty_factor":{turn_penalty_factor}"""+"}"
    meili_request_body = meili_head + meili_coordinates + meili_tail
    # url = "http://localhost:8002/trace_route"
    url = "http://localhost:8002/trace_attributes"
    headers = {'Content-type': 'application/json'}
    request = requests.post(url, data=str(meili_request_body), headers=headers)

        # READ & FORMAT VALHALLA RESPONSE
    if request.status_code == 200:
        response_text = json.loads(request.text)
    else:
        print(f'API Error, {request.status_code}')
        return None, pd.DataFrame()

    matching = dict(response_text).get('shape')

    # lst_MapMatchingRoute = [LineString(decode(matching))]
    # estimated_route = gpd.GeoDataFrame(geometry=lst_MapMatchingRoute, crs=4326)
    lst_MapMatchingRoute = LineString(decode(matching))

    tracepoints = list(response_text.get('matched_points'))
    df_mapmatchedGPS_points = pd.DataFrame([(p['lon'],p['lat']) if (p is not None) else [None,None] for p in tracepoints] , columns=['LongitudeMatched', 'LatitudeMatched']).reset_index(drop=True)

    # result = pd.concat([data, df_mapmatchedGPS_points], axis=1)

    # df_mapmatchedGPS_points = df_mapmatchedGPS_points.loc[
    #     (df_mapmatchedGPS_points['lat'].notnull()) &
    #     (df_mapmatchedGPS_points['lon'].notnull())]
    
    return lst_MapMatchingRoute, df_mapmatchedGPS_points

# Decluttering

In [7]:
# Window length: Controls the precision
w=6
# Radius (m): Controls the potential GPS error
r=50
# Sensitity: Control the spread
s=1

In [229]:
csv_file = '../data/processed/trips/backup/70_20240729.csv'
data = pd.read_csv(csv_file)
data['ValidUntilTime'] = pd.to_datetime(data['ValidUntilTime'])
data['RecordedAtTime'] = pd.to_datetime(data['RecordedAtTime'])
# data['EpochTime'] = data['RecordedAtTime'].apply(lambda x: x.timestamp())
data['Longitude'] = pd.to_numeric(data['Longitude'])
data['Latitude'] = pd.to_numeric(data['Latitude'])
# data['HexId'] = data.apply(lambda x: h3.geo_to_h3(x['Latitude'], x['Longitude'], resolution=hex_res),
#                                 axis=1) # Row wise operation
data.sort_values(by=['RecordedAtTime', 'ValidUntilTime', 'DatedVehicleJourneyRef', 'VehicleUniqueId', 'LineRef'], 
                    ascending=True, ignore_index=True, inplace=True)
data.drop_duplicates(subset=['RecordedAtTime', 'LineRef', 'DirectionRef', 'VehicleUniqueId', 'BlockRef', 'OriginAimedDepartureTime', 'DestinationAimedArrivalTime', 'DatedVehicleJourneyRef'], 
                        keep='first', inplace=True, ignore_index=True)

# sample_data = data.loc[
#     (data['LineRef'].astype(str)=='m1') &
#     (data['DirectionRef'].astype(str)=='inbound') &
#     # (str(data['OriginRef'])=='0100FBX18342') &
#     # (str(data['OriginAimedDepartureTime'])=='2024-07-14T20:55:00+00:00') &
#     # (str(data['BlockRef'])=='7001') &
#     (data['VehicleRef'].astype(str)=='FBRI-39335') &
#     (data['DatedVehicleJourneyRef'].astype(str)=='2300')   
# ]

# sample_data = data.loc[
#     (data['TripId'].astype(str)=='6f9e820d-bc64-33d6-c2e8-d3c12478febb')
# ]

trip_list = data['TripId'].unique().tolist()
sample_data = data.loc[
    (data['TripId'].astype(str)==trip_list[24])
]

uncluttered_set = decluttering_data(sample_data, w=6,r=50,s=1,lon_col_name='Longitude',lat_col_name='Latitude')
print(sample_data.shape)
print(uncluttered_set.shape)
print(sample_data['TripId'].iloc[0])
# uncluttered_set.head()

(215, 23)
(186, 23)
fdff693a-8e32-f394-c965-b5260ecaed45


In [195]:
# Snippet for checking anomaly
geod = Geod(ellps="WGS84")
for i in range(0,len(trip_list)-1):
    sample_data = data.loc[
        (data['TripId'].astype(str)==trip_list[i])
    ]
    uncluttered_set = decluttering_data(sample_data, w=6,r=50,s=1,lon_col_name='Longitude',lat_col_name='Latitude')
    coor_raw_data = pd.DataFrame({
        'lon': sample_data['Longitude'],
        'lat': sample_data['Latitude'],
        'time': sample_data['RecordedAtTime'].apply(lambda x: x.timestamp())
    }).reset_index(drop=True)

    uncluttered_data = pd.DataFrame({
        'lon': uncluttered_set['Longitude'],
        'lat': uncluttered_set['Latitude'],
        'time': uncluttered_set['RecordedAtTime'].apply(lambda x: x.timestamp())
    }).reset_index(drop=True)

    lst_MapMatchingRoute, matched_ucluttered = map_matching(uncluttered_set, 'Longitude', 'Latitude', 'RecordedAtTime')
    og_lst_MapMatchingRoute, og_matched = map_matching(sample_data, 'Longitude', 'Latitude', 'RecordedAtTime')
    if geod.geometry_length(og_lst_MapMatchingRoute) - geod.geometry_length(lst_MapMatchingRoute) > 800:
    # if (sample_data.shape[0] - uncluttered_set.shape[0] > 50) and (uncluttered_set.shape[0] > 100):
        print(i)

24


In [230]:
coor_raw_data = pd.DataFrame({
    'lon': sample_data['Longitude'],
    'lat': sample_data['Latitude'],
    'time': sample_data['RecordedAtTime'].apply(lambda x: x.timestamp())
}).reset_index(drop=True)

uncluttered_data = pd.DataFrame({
    'lon': uncluttered_set['Longitude'],
    'lat': uncluttered_set['Latitude'],
    'time': uncluttered_set['RecordedAtTime'].apply(lambda x: x.timestamp())
}).reset_index(drop=True)

# meili_coordinates = uncluttered_data.to_json(orient='records')
# meili_head = '{"shape":'
# meili_tail = ""","search_radius": 5, "shape_match":"map_snap", "costing":"bus", "format":"osrm", "turn_penalty_factor":100}"""
# meili_request_body = meili_head + meili_coordinates + meili_tail
# # url = "http://localhost:8002/trace_route"
# url = "http://localhost:8002/trace_attributes"
# headers = {'Content-type': 'application/json'}
# request = requests.post(url, data=str(meili_request_body), headers=headers)

#     #%% READ & FORMAT VALHALLA RESPONSE
# if request.status_code == 200:
#     response_text = json.loads(request.text)
# else:
#     print(f'API Error, {r.status_code}')
# #     # return lon_list, lat_list
# matching = dict(response_text).get('shape')

# lst_MapMatchingRoute = [LineString(decode(matching))]
# estimated_route = gpd.GeoDataFrame(geometry=lst_MapMatchingRoute, crs=4326)

# tracepoints = list(response_text.get('matched_points'))
# df_mapmatchedGPS_points = pd.DataFrame([(p['lon'],p['lat']) if (p is not None) else [None,None] for p in tracepoints] , columns=['lon', 'lat'])
# df_mapmatchedGPS_points = df_mapmatchedGPS_points.loc[
#     (df_mapmatchedGPS_points['lat'].notnull()) &
#     (df_mapmatchedGPS_points['lon'].notnull())
# ]

lst_MapMatchingRoute, matched_ucluttered = map_matching(uncluttered_set, 'Longitude', 'Latitude', 'RecordedAtTime', )
estimated_route = gpd.GeoDataFrame(geometry=[lst_MapMatchingRoute], crs=4326)
df_mapmatchedGPS_points = matched_ucluttered.loc[
    (matched_ucluttered['LatitudeMatched'].notnull()) &
    (matched_ucluttered['LongitudeMatched'].notnull())
]

og_lst_MapMatchingRoute, og_matched = map_matching(sample_data, 'Longitude', 'Latitude', 'RecordedAtTime')
og_estimated_route = gpd.GeoDataFrame(geometry=[og_lst_MapMatchingRoute], crs=4326)
og_df_mapmatchedGPS_points = og_matched.loc[
    (og_matched['LatitudeMatched'].notnull()) &
    (og_matched['LongitudeMatched'].notnull())
]

str(lst_MapMatchingRoute)

'LINESTRING (-2.544947 51.499261, -2.544927 51.499194, -2.544934 51.499155, -2.544955 51.499118, -2.544985 51.499083, -2.54503 51.499043, -2.545083 51.499013, -2.545142 51.498991, -2.545205 51.498975, -2.545265 51.498966, -2.545344 51.498962, -2.545419 51.498969, -2.545496 51.498985, -2.545568 51.499008, -2.545646 51.499049, -2.545688 51.49909, -2.545709 51.499126, -2.545766 51.499297, -2.545835 51.499427, -2.545867 51.499509, -2.546039 51.500116, -2.546076 51.500335, -2.546085 51.500391, -2.546074 51.500433, -2.546037 51.500499, -2.545981 51.500554, -2.545932 51.500589, -2.545864 51.500618, -2.545772 51.500635, -2.545684 51.500639, -2.545623 51.500638, -2.545526 51.500619, -2.545449 51.500591, -2.545358 51.500545, -2.545322 51.500515, -2.545295 51.500479, -2.545251 51.50037, -2.545234 51.500354, -2.545199 51.500324, -2.545135 51.500289, -2.545049 51.500259, -2.544965 51.500245, -2.544928 51.500244, -2.544891 51.500242, -2.544799 51.500248, -2.544719 51.500261, -2.544693 51.500268, -2.

In [233]:
geod = Geod(ellps="WGS84")

before_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(coor_raw_data['lon'], coor_raw_data['lat']), crs=4326)
before_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in coor_raw_data[['lon', 'lat']].itertuples(index=False, name=None)])], crs=4326)
before_all = gpd.GeoDataFrame(pd.concat([before_linestring, before_points], ignore_index=True))

before_d_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(uncluttered_data['lon'], uncluttered_data['lat']), crs=4326)
before_d_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in uncluttered_data[['lon', 'lat']].itertuples(index=False, name=None)])], crs=4326)
before_d_all = gpd.GeoDataFrame(pd.concat([before_d_linestring, before_d_points], ignore_index=True))

after_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_mapmatchedGPS_points['LongitudeMatched'], df_mapmatchedGPS_points['LatitudeMatched']), crs=4326)
after_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in df_mapmatchedGPS_points[['LongitudeMatched', 'LatitudeMatched']].itertuples(index=False, name=None)])], crs=4326)
after_all = gpd.GeoDataFrame(pd.concat([after_linestring, after_points], ignore_index=True))

og_after_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(og_df_mapmatchedGPS_points['LongitudeMatched'], og_df_mapmatchedGPS_points['LatitudeMatched']), crs=4326)
og_after_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in og_df_mapmatchedGPS_points[['LongitudeMatched', 'LatitudeMatched']].itertuples(index=False, name=None)])], crs=4326)
og_after_all = gpd.GeoDataFrame(pd.concat([og_after_points, og_after_linestring], ignore_index=True))


#%% RAW & MAP-MATCHING ROUTES - DRAW MAP
f = folium.Figure(800,800)
m = folium.Map([51.49856419519174, -2.548053188420676], tiles='cartodbdark_matter', zoom_start=15).add_to(f)
# m = folium.Map([51.49856419519174, -2.548053188420676], tiles='OpenStreetMap', zoom_start=14)
# folium.GeoJson(before_all, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='rawGPS_points').add_to(m)
folium.GeoJson(after_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=0.7), name='snapped_points').add_to(m)
# folium.GeoJson(estimated_route, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)
# folium.GeoJson(og_estimated_route, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='og_estimated_route').add_to(m)
# folium.GeoJson(estimated_route, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)

folium.GeoJson(before_points, style_function=lambda x:{'color': 'blue', 'opacity': 0.5}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=0.7), name='rawGPS_points').add_to(m)
# folium.GeoJson(before_d_points, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='rawGPS_points_decluttered').add_to(m)
# folium.GeoJson(after_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='snapped_points').add_to(m)
# folium.GeoJson(estimated_route, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)


# folium.CircleMarker(location=[51.45837, -2.59167],
#                         radius=10,
#                         color='yellow',
#                         weight=5).add_to(m)

# folium.CircleMarker(location=[51.458433, -2.591871],
#                         radius=10,
#                         color='yellow',
#                         weight=5).add_to(m)


folium.LayerControl(position='topright', collapsed=False).add_to(m)
# m.save('mapmatching.html')

print(f'Route length from point-to-point original GPS data (m): {round(geod.geometry_length(before_linestring["geometry"][0]), 2)}')
print(f'Route length from map-matched GPS data (m): {round(geod.geometry_length(og_lst_MapMatchingRoute),2)}')
print(f'Route length from map-macthed and decluttered GPS data (m): {round(geod.geometry_length(lst_MapMatchingRoute),2)}')
m

Route length from point-to-point original GPS data (m): 18627.03
Route length from map-matched GPS data: 20197.22
Route length from map-macthed and decluttered GPS data: 18780.86


# Line String

In [121]:
def p2p_distance(line, point1, point2):
    '''
    enter a Linestring, and 2 Point shapely object
    returns the distance of two points on the line
    '''
    # Project the points onto the LineString to get distances along the LineString
    proj1 = line.project(point1)
    proj2 = line.project(point2)
    
    # Ensure proj1 is less than proj2 for correct segment extraction
    start_proj, end_proj = sorted([proj1, proj2])
    
    # line_gpd = gpd.GeoDataFrame(geometry=[line], crs=4326)
    geod = Geod(ellps="WGS84")
    line_length = geod.geometry_length(line)
    
    return (end_proj-start_proj)*line_length


# Example usage
line = LineString([(-2.588711,51.432328),(-2.587898,51.432914),(-2.587241,51.433421),(-2.587437,51.433539),(-2.587548,51.433615),(-2.587694,51.433725),(-2.587821,51.433837),(-2.587935,51.433953),(-2.588057,51.43408),(-2.588122,51.434168),(-2.58828,51.434414),(-2.588326,51.43451),(-2.588363,51.434596),(-2.588429,51.434755),(-2.58848,51.434881),(-2.588575,51.435038),(-2.58868,51.435185),(-2.589087,51.435803),(-2.589271,51.43606),(-2.589293,51.436096),(-2.589366,51.436227),(-2.589387,51.436341),(-2.589404,51.436435),(-2.5901,51.436431),(-2.590862,51.436415),(-2.592166,51.436375),(-2.592593,51.436364),(-2.592818,51.436348),(-2.593018,51.436317),(-2.593464,51.436194),(-2.59348,51.43619),(-2.593782,51.436098),(-2.594105,51.436014),(-2.594528,51.435903),(-2.594935,51.435809),(-2.595029,51.435796),(-2.595257,51.435765),(-2.59628,51.435635),(-2.596662,51.43559),(-2.597033,51.435567),(-2.597321,51.435579),(-2.597679,51.435611),(-2.597736,51.435613),(-2.597765,51.435608),(-2.597792,51.435603),(-2.597839,51.435601),(-2.597889,51.435609),(-2.597932,51.435626),(-2.597961,51.435651),(-2.597979,51.435686),(-2.597977,51.435713),(-2.597971,51.435724),(-2.597963,51.435739),(-2.598071,51.435882),(-2.598257,51.436071),(-2.598491,51.436353),(-2.598584,51.436484),(-2.598687,51.436622),(-2.598721,51.436678),(-2.598802,51.436777),(-2.598822,51.436802),(-2.598931,51.436898),(-2.599062,51.437004),(-2.599138,51.437064),(-2.599378,51.437196),(-2.599561,51.437319),(-2.599579,51.43733),(-2.599699,51.43741),(-2.599845,51.437508),(-2.59993,51.437564),(-2.600098,51.437732),(-2.60017,51.437817),(-2.600208,51.437912),(-2.600243,51.438032),(-2.600256,51.43825),(-2.600267,51.43833),(-2.600273,51.438371),(-2.600283,51.438435),(-2.600302,51.438566),(-2.600315,51.438612),(-2.600424,51.438996),(-2.600475,51.439086),(-2.600839,51.439651),(-2.600807,51.439717),(-2.600793,51.439791),(-2.600803,51.439856),(-2.600838,51.439939),(-2.60084,51.439963),(-2.600831,51.440082),(-2.600856,51.440343),(-2.60086,51.440456),(-2.600842,51.440555),(-2.600813,51.440631),(-2.600766,51.440717),(-2.600751,51.440791),(-2.600648,51.440813),(-2.600211,51.440922),(-2.599729,51.441038),(-2.599296,51.441142),(-2.598752,51.441273),(-2.598678,51.441293),(-2.598423,51.44139),(-2.598218,51.441512),(-2.598037,51.441605),(-2.597706,51.441774),(-2.597204,51.441897),(-2.597109,51.441944),(-2.596963,51.441991),(-2.596577,51.442093),(-2.596389,51.442154),(-2.596284,51.442188),(-2.595893,51.442315),(-2.595763,51.442356),(-2.595716,51.442403),(-2.595703,51.442414),(-2.595678,51.442432),(-2.595591,51.442495),(-2.5954,51.44264),(-2.595326,51.442695),(-2.594698,51.443199),(-2.594438,51.443394),(-2.594087,51.443672),(-2.594044,51.443702),(-2.593842,51.443844),(-2.593726,51.443915),(-2.593549,51.444046),(-2.593329,51.444235),(-2.593167,51.444374),(-2.592725,51.444748),(-2.592647,51.444809),(-2.592588,51.444848),(-2.592499,51.44489),(-2.592305,51.44496),(-2.592036,51.445051),(-2.591978,51.445072),(-2.591936,51.445093),(-2.591908,51.445111),(-2.591871,51.44514),(-2.591844,51.44517),(-2.591815,51.445227),(-2.591797,51.445289),(-2.591776,51.445409),(-2.591772,51.445447),(-2.591763,51.445491),(-2.591744,51.445542),(-2.59172,51.44559),(-2.591705,51.44561),(-2.591535,51.445803),(-2.591459,51.445888),(-2.591422,51.445928),(-2.591405,51.445939),(-2.591389,51.445956),(-2.591358,51.445977),(-2.591341,51.445988),(-2.591267,51.446021),(-2.591252,51.446025),(-2.591198,51.446065),(-2.591149,51.446105),(-2.591107,51.446143),(-2.59108,51.446169),(-2.591066,51.446188),(-2.59105,51.446209),(-2.590998,51.446311),(-2.590986,51.446351),(-2.590971,51.446426),(-2.590921,51.446705),(-2.590914,51.446737),(-2.590881,51.446905),(-2.590855,51.447026),(-2.590827,51.447132),(-2.590794,51.447241),(-2.590773,51.447308),(-2.590754,51.447358),(-2.590684,51.447543),(-2.590517,51.448214),(-2.590432,51.448596),(-2.59043,51.448626),(-2.590438,51.448666),(-2.590452,51.448706),(-2.590474,51.448745),(-2.590508,51.448794),(-2.590552,51.448824),(-2.59061,51.448886),(-2.590649,51.448954),(-2.590756,51.449064),(-2.590796,51.44912),(-2.590832,51.449158),(-2.590872,51.449189),(-2.590913,51.449214),(-2.590968,51.44924),(-2.591048,51.449275),(-2.591106,51.449293),(-2.591333,51.449362),(-2.591355,51.449367),(-2.591642,51.449401),(-2.592473,51.449518),(-2.592658,51.449555),(-2.592782,51.44956),(-2.592858,51.449477),(-2.592896,51.449421),(-2.592946,51.449361),(-2.593018,51.449333),(-2.593108,51.44932),(-2.593325,51.44931),(-2.593521,51.449311),(-2.593598,51.449313),(-2.593757,51.449325),(-2.594014,51.449342),(-2.594207,51.449355),(-2.594861,51.449399),(-2.595065,51.449413),(-2.59531,51.44943),(-2.59648,51.44951),(-2.596821,51.449527),(-2.596814,51.449659),(-2.596796,51.449719),(-2.596724,51.44996),(-2.596601,51.450412),(-2.596534,51.450663),(-2.596486,51.450841),(-2.596464,51.450921),(-2.596475,51.450968),(-2.596469,51.451057),(-2.596388,51.45137),(-2.596362,51.45149),(-2.596355,51.45154),(-2.59644,51.451662),(-2.596496,51.451737),(-2.596553,51.451803),(-2.5966,51.451865),(-2.59663,51.451901),(-2.596686,51.45196),(-2.596746,51.452018),(-2.596847,51.452113),(-2.596981,51.452279),(-2.597063,51.452417),(-2.597115,51.452552),(-2.597137,51.452776),(-2.597116,51.453408),(-2.597115,51.453439),(-2.597113,51.453491),(-2.597076,51.453716),(-2.596999,51.453897),(-2.596821,51.454253),(-2.596796,51.454308),(-2.596821,51.454346),(-2.596827,51.454356),(-2.596845,51.454401),(-2.596848,51.454424),(-2.596884,51.454469),(-2.596968,51.454548),(-2.596999,51.454597),(-2.597011,51.454643),(-2.597013,51.454725),(-2.596497,51.455264),(-2.596403,51.455371),(-2.596335,51.455502),(-2.596244,51.455709),(-2.596128,51.45611),(-2.596093,51.456286),(-2.596037,51.456477),(-2.595963,51.456659),(-2.595783,51.456925),(-2.595653,51.457029),(-2.59557,51.457087),(-2.595404,51.457188),(-2.595274,51.457264),(-2.595155,51.45732),(-2.595024,51.457379),(-2.594823,51.457446),(-2.593444,51.457807),(-2.593091,51.457887),(-2.593027,51.4579),(-2.592934,51.457916),(-2.592761,51.457942),(-2.592622,51.457961),(-2.592482,51.457992),(-2.592385,51.458022),(-2.592296,51.458058),(-2.592226,51.458101),(-2.592161,51.45816),(-2.592069,51.458246),(-2.591598,51.458666),(-2.591361,51.458884),(-2.591308,51.458948),(-2.591283,51.45901),(-2.591275,51.459065),(-2.591285,51.459193),(-2.591353,51.459261),(-2.591383,51.459333),(-2.591375,51.459456),(-2.591334,51.459539),(-2.591211,51.459646),(-2.59114,51.45967),(-2.590901,51.459835),(-2.590862,51.459867),(-2.590851,51.459894),(-2.590844,51.459943),(-2.590854,51.460017),(-2.590886,51.460114),(-2.590893,51.46013),(-2.591111,51.460477),(-2.591142,51.460526),(-2.591153,51.460622),(-2.59115,51.460715),(-2.591101,51.460835),(-2.590999,51.460984),(-2.590956,51.460996),(-2.590856,51.461139),(-2.590726,51.461362),(-2.590735,51.461416),(-2.590709,51.461458),(-2.590689,51.46149),(-2.590624,51.461525),(-2.590556,51.461631),(-2.590459,51.461791),(-2.590398,51.461896),(-2.590339,51.462004),(-2.590309,51.462058),(-2.590299,51.462121),(-2.590262,51.462193),(-2.590186,51.462313),(-2.590024,51.462635),(-2.59,51.462694),(-2.589989,51.462723),(-2.589956,51.462746),(-2.589954,51.462855),(-2.589942,51.462913),(-2.589871,51.463239),(-2.58976,51.463797),(-2.589744,51.463992),(-2.589736,51.464093),(-2.589729,51.464229),(-2.58982,51.464397),(-2.590307,51.465332),(-2.59043,51.465562),(-2.590501,51.465592),(-2.590507,51.465601),(-2.590542,51.465669),(-2.590601,51.465776),(-2.590636,51.465843),(-2.590665,51.465901),(-2.59069,51.465963),(-2.590676,51.466017),(-2.590763,51.466176),(-2.590923,51.466512),(-2.591059,51.46678),(-2.591247,51.467129),(-2.591367,51.467379),(-2.591431,51.467496),(-2.591572,51.467691),(-2.591641,51.4678),(-2.591946,51.468099),(-2.592181,51.468289),(-2.592197,51.468302),(-2.592816,51.468792),(-2.592879,51.468847),(-2.592902,51.468864),(-2.592978,51.468912),(-2.592916,51.468947),(-2.592882,51.469091),(-2.592831,51.469298),(-2.592776,51.469431),(-2.592715,51.469483),(-2.592655,51.469517),(-2.592573,51.469543),(-2.592526,51.46955),(-2.592481,51.469548),(-2.591124,51.469285),(-2.59066,51.469169),(-2.590219,51.469035),(-2.589793,51.468905),(-2.589353,51.468799),(-2.589048,51.468754),(-2.588752,51.468721),(-2.588507,51.468728),(-2.588265,51.468762),(-2.587999,51.468815),(-2.587433,51.468937),(-2.58659,51.469127),(-2.585996,51.469285),(-2.585805,51.469364),(-2.585635,51.469466),(-2.584964,51.470001),(-2.584851,51.470149),(-2.584829,51.470203),(-2.584821,51.470246),(-2.584824,51.47026),(-2.584855,51.470317),(-2.585158,51.4709),(-2.585194,51.470991),(-2.585194,51.471013),(-2.585187,51.471034),(-2.585164,51.47108),(-2.585123,51.471127),(-2.585061,51.471167),(-2.584405,51.47154),(-2.583102,51.472282),(-2.583047,51.472316),(-2.582382,51.472691),(-2.582006,51.472905),(-2.581984,51.472948),(-2.581964,51.472958),(-2.58191,51.472989),(-2.581906,51.472998),(-2.581899,51.473006),(-2.58189,51.473122),(-2.581891,51.473139),(-2.581891,51.473146),(-2.581889,51.473151),(-2.581886,51.473156),(-2.581881,51.47316),(-2.581861,51.473171),(-2.581852,51.473256),(-2.58183,51.473318),(-2.581491,51.473983),(-2.58141,51.474123),(-2.581373,51.474184),(-2.581379,51.47422),(-2.581369,51.474239),(-2.581357,51.474263),(-2.581316,51.474286),(-2.581045,51.4748),(-2.580965,51.47495),(-2.580932,51.475013),(-2.580871,51.475116),(-2.580776,51.475287),(-2.580494,51.475824),(-2.580302,51.476158),(-2.58023,51.476288),(-2.580123,51.476472),(-2.580069,51.476663),(-2.580021,51.476883),(-2.579954,51.477186),(-2.579887,51.477406),(-2.579701,51.47801),(-2.57964,51.478293),(-2.579643,51.478362),(-2.579703,51.478458),(-2.580041,51.478772),(-2.580216,51.478938),(-2.580819,51.479507),(-2.58095,51.479619),(-2.58112,51.479777),(-2.581391,51.48003),(-2.581539,51.480157),(-2.581899,51.480473),(-2.582009,51.480567),(-2.582025,51.48058),(-2.580428,51.481324),(-2.580222,51.481421),(-2.579628,51.481703),(-2.578801,51.482082),(-2.578948,51.482248),(-2.579009,51.482324),(-2.579077,51.482408),(-2.579147,51.482499),(-2.579209,51.482599),(-2.579332,51.482867),(-2.579407,51.483118),(-2.579473,51.483617),(-2.579563,51.485144),(-2.579561,51.485178),(-2.579568,51.485313),(-2.57958,51.485415),(-2.579582,51.485448),(-2.579608,51.485734),(-2.579637,51.48592),(-2.579644,51.485962),(-2.579663,51.48608),(-2.579715,51.486263),(-2.579724,51.486285),(-2.579842,51.486553),(-2.57987,51.4866),(-2.580105,51.486983),(-2.580401,51.487335),(-2.580577,51.487491),(-2.580797,51.487698),(-2.581004,51.487845),(-2.581187,51.487962),(-2.581504,51.488126),(-2.581846,51.48828),(-2.581897,51.488302),(-2.582137,51.488419),(-2.581899,51.488531),(-2.581849,51.488556),(-2.581594,51.488684),(-2.581429,51.488769),(-2.581299,51.488843),(-2.580728,51.489169),(-2.580601,51.489237),(-2.580475,51.48931),(-2.580216,51.489455),(-2.579773,51.489699),(-2.579564,51.489811),(-2.579325,51.489944),(-2.579113,51.490056),(-2.578862,51.490187),(-2.578763,51.490247),(-2.578647,51.490322),(-2.578461,51.490465),(-2.578327,51.490579),(-2.578208,51.490684),(-2.578091,51.490782),(-2.577974,51.490883),(-2.577786,51.491041),(-2.577718,51.491097),(-2.577642,51.491159),(-2.577551,51.491238),(-2.57748,51.491298),(-2.577364,51.491398),(-2.577034,51.491682),(-2.576964,51.491743),(-2.576941,51.491766),(-2.576873,51.491823),(-2.576731,51.491953),(-2.576403,51.492252),(-2.576329,51.492326),(-2.575956,51.492696),(-2.575746,51.492916),(-2.575723,51.492941),(-2.575564,51.493109),(-2.575391,51.493298),(-2.575286,51.49342),(-2.575175,51.493546),(-2.575091,51.493653),(-2.5747,51.494145),(-2.574541,51.494358),(-2.574471,51.494458),(-2.574401,51.494557),(-2.574254,51.494768),(-2.574148,51.494929),(-2.57398,51.495165),(-2.573875,51.495321),(-2.573812,51.495423),(-2.573759,51.495508),(-2.573483,51.495926),(-2.573456,51.495968),(-2.573197,51.496367),(-2.573118,51.496502),(-2.572999,51.496713),(-2.572969,51.496766),(-2.57284,51.496988),(-2.57279,51.497075),(-2.57268,51.497257),(-2.572588,51.497415),(-2.572531,51.497511),(-2.572373,51.497806),(-2.572291,51.497963),(-2.572214,51.498097),(-2.572095,51.498301),(-2.572025,51.498424),(-2.571972,51.49852),(-2.571745,51.498924),(-2.571625,51.499131),(-2.571509,51.499341),(-2.571269,51.499749),(-2.570992,51.500166),(-2.570322,51.500992),(-2.569789,51.501476),(-2.569662,51.50159),(-2.569456,51.501769),(-2.569241,51.501952),(-2.5685,51.50259),(-2.567751,51.503237),(-2.567577,51.503388),(-2.567436,51.503506),(-2.567294,51.503621),(-2.566693,51.50417),(-2.56647,51.504468),(-2.56633,51.504766),(-2.566263,51.505043),(-2.566195,51.505441),(-2.566019,51.505861),(-2.565794,51.506231),(-2.565502,51.506553),(-2.56507,51.50694),(-2.564736,51.507257),(-2.564486,51.507614),(-2.564405,51.50773),(-2.56433,51.50784),(-2.564283,51.507953),(-2.564205,51.507997),(-2.564175,51.508009),(-2.56414,51.508024),(-2.563983,51.508056),(-2.563687,51.508074),(-2.563439,51.507994),(-2.563138,51.507913),(-2.563079,51.507895),(-2.562935,51.507858),(-2.562615,51.507776),(-2.562576,51.507767),(-2.562499,51.507749),(-2.562284,51.507707),(-2.562202,51.507695),(-2.56,51.507453),(-2.558645,51.507314),(-2.558497,51.507296),(-2.55781,51.507228),(-2.557688,51.507217),(-2.557145,51.507167),(-2.556614,51.50711),(-2.555837,51.506997),(-2.555696,51.507042),(-2.555502,51.507048),(-2.555349,51.50706),(-2.555249,51.507063),(-2.555001,51.507077),(-2.5549,51.507086),(-2.554799,51.507074),(-2.554638,51.507027),(-2.554544,51.506969),(-2.554453,51.506908),(-2.554371,51.506807),(-2.554378,51.506696),(-2.554402,51.506619),(-2.554598,51.506327),(-2.554581,51.506243),(-2.554617,51.506197),(-2.55475,51.506082),(-2.554752,51.506081),(-2.554824,51.506007),(-2.554953,51.50591),(-2.555023,51.505854),(-2.555154,51.505747),(-2.555258,51.505604),(-2.555316,51.505458),(-2.555357,51.505119),(-2.55534,51.504883),(-2.555351,51.504788),(-2.555344,51.504673),(-2.555337,51.50457),(-2.555267,51.504451),(-2.55515,51.504347),(-2.555055,51.504311),(-2.554985,51.504267),(-2.554944,51.504215),(-2.554912,51.504156),(-2.554908,51.504081),(-2.554936,51.504015),(-2.555006,51.503943),(-2.555069,51.503904),(-2.555183,51.503799),(-2.555244,51.503652),(-2.555216,51.503452),(-2.555145,51.503249),(-2.554978,51.502992),(-2.554785,51.502761),(-2.554673,51.502554),(-2.554644,51.50238),(-2.554658,51.502198),(-2.554721,51.502046),(-2.554865,51.501861),(-2.555067,51.501665),(-2.555325,51.501441),(-2.555527,51.50129),(-2.555771,51.501166),(-2.556062,51.501044),(-2.556839,51.500754),(-2.556879,51.500735),(-2.556915,51.500716),(-2.556957,51.500693),(-2.55699,51.500673),(-2.55707,51.50062),(-2.557074,51.500572),(-2.557102,51.500527),(-2.55715,51.500488),(-2.557212,51.500462),(-2.557284,51.500448),(-2.557359,51.500447),(-2.557431,51.50046),(-2.557494,51.500486),(-2.557543,51.500522),(-2.557718,51.500585),(-2.5578,51.500609),(-2.557876,51.500627),(-2.557943,51.500638),(-2.55811,51.500666),(-2.558561,51.500699),(-2.558772,51.5007),(-2.559062,51.500685),(-2.559436,51.500616),(-2.559742,51.500502),(-2.55986,51.500428),(-2.559731,51.500336),(-2.55952,51.500144),(-2.559435,51.500048),(-2.559379,51.499981),(-2.559335,51.49989),(-2.559295,51.499716),(-2.559223,51.499477),(-2.559102,51.499225),(-2.558942,51.498962),(-2.558792,51.498793),(-2.558678,51.498682),(-2.558401,51.498464),(-2.558214,51.498344),(-2.558572,51.498053),(-2.55862,51.498016),(-2.558709,51.497994),(-2.558777,51.497994),(-2.558811,51.498008),(-2.559028,51.497934),(-2.559192,51.497872),(-2.559502,51.497761),(-2.559529,51.497751),(-2.559722,51.497687),(-2.55987,51.497638),(-2.56001,51.497591),(-2.560269,51.49752),(-2.560262,51.497491),(-2.560188,51.497375),(-2.560116,51.497305),(-2.560027,51.497246),(-2.5599,51.497177),(-2.559755,51.497114),(-2.559678,51.497101),(-2.559596,51.497095),(-2.559535,51.49708),(-2.559411,51.497026),(-2.559221,51.496937),(-2.559158,51.49688),(-2.559107,51.496796),(-2.559023,51.496736),(-2.558948,51.496698),(-2.558889,51.496674),(-2.558802,51.496653),(-2.558713,51.496639),(-2.558457,51.496641),(-2.558329,51.496645),(-2.558232,51.496659),(-2.558048,51.496706),(-2.557787,51.496728),(-2.557334,51.496753),(-2.556913,51.496751),(-2.55664,51.496741),(-2.556364,51.496734),(-2.556178,51.496726),(-2.556017,51.496701),(-2.555712,51.496602),(-2.555519,51.496566),(-2.555482,51.496564),(-2.555359,51.496557),(-2.555157,51.496562),(-2.55499,51.496583),(-2.554873,51.496607),(-2.554695,51.496657),(-2.554573,51.496695),(-2.554457,51.496718),(-2.554318,51.496732),(-2.554032,51.496737),(-2.553742,51.496742),(-2.553462,51.496758),(-2.553412,51.496762),(-2.553127,51.496788),(-2.552834,51.496814),(-2.552664,51.496829),(-2.552274,51.496865),(-2.551961,51.496894),(-2.551763,51.496911),(-2.551413,51.496941),(-2.551075,51.496972),(-2.550665,51.497009),(-2.550263,51.497049),(-2.550153,51.497058),(-2.549872,51.497081),(-2.549698,51.49709),(-2.549493,51.497097),(-2.54936,51.497082),(-2.549268,51.497069),(-2.54917,51.49705),(-2.549076,51.497034),(-2.548984,51.497021),(-2.548938,51.497016),(-2.548905,51.497012),(-2.548782,51.496998),(-2.548658,51.496991),(-2.54853,51.49699),(-2.548367,51.496991),(-2.548197,51.497),(-2.548021,51.497017),(-2.547844,51.497043),(-2.547679,51.497073),(-2.54752,51.49711),(-2.547354,51.497153),(-2.547198,51.497193),(-2.547035,51.497234),(-2.546751,51.497306),(-2.546589,51.497348),(-2.546458,51.497379),(-2.546292,51.497412),(-2.546017,51.497455),(-2.54579,51.497486),(-2.545542,51.497507),(-2.545341,51.49752),(-2.545109,51.497535),(-2.54493,51.497545),(-2.544744,51.497561),(-2.544532,51.497587),(-2.544209,51.497664),(-2.544014,51.497718),(-2.543845,51.497782),(-2.543814,51.497795),(-2.543701,51.497843),(-2.543761,51.497986),(-2.543802,51.498107),(-2.543841,51.498265),(-2.543869,51.498429),(-2.543882,51.498489),(-2.543891,51.498554),(-2.543926,51.499225),(-2.543942,51.499614),(-2.544,51.499728),(-2.544049,51.49979),(-2.544097,51.499839),(-2.544151,51.499883),(-2.544218,51.499937),(-2.54433,51.500014),(-2.544428,51.500073),(-2.544651,51.500118),(-2.544701,51.500128),(-2.544754,51.500137),(-2.544813,51.50014),(-2.54486,51.500136),(-2.544896,51.500128),(-2.544929,51.500117),(-2.544961,51.500103),(-2.545017,51.50007),(-2.545044,51.500049),(-2.545069,51.500027),(-2.545093,51.500002),(-2.545113,51.499974),(-2.545132,51.499937),(-2.545036,51.499568),(-2.544927,51.499194),(-2.544934,51.499155),(-2.544955,51.499118),(-2.544985,51.499083),(-2.54503,51.499043),(-2.545083,51.499013),(-2.545142,51.498991),(-2.545205,51.498975),(-2.545265,51.498966),(-2.545344,51.498962),(-2.545419,51.498969),(-2.545496,51.498985),(-2.545568,51.499008),(-2.545646,51.499049),(-2.545688,51.49909),(-2.545709,51.499126),(-2.545766,51.499297),(-2.545835,51.499427),(-2.545867,51.499509),(-2.545939,51.499763)])
point1 = Point(-2.599783, 51.437467)
point2 = Point(-2.596805, 51.449687)

sub_line = create_sub_linestring(line, point1, point2)
sub_line

383.54763439786433

In [118]:
proj1 = line.project(point1)
proj2 = line.project(point2)

In [119]:
proj1

0.016914773258813

In [66]:
point1_on_line = line.interpolate(proj1)
point2_on_line = line.interpolate(proj2)

In [78]:
point1_gpd = gpd.GeoDataFrame(geometry=[point1_on_line, point2_on_line], crs=4326)


In [114]:
line_gpd = gpd.GeoDataFrame(geometry=[line], crs=4326)
geod = Geod(ellps="WGS84")
line_length = geod.geometry_length(line_gpd['geometry'][0])

In [115]:
line_length

15552.655671887824

In [74]:
# before_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(coor_raw_data['lon'], coor_raw_data['lat']), crs=4326)
# before_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in coor_raw_data[['lon', 'lat']].itertuples(index=False, name=None)])], crs=4326)
# before_all = gpd.GeoDataFrame(pd.concat([before_linestring, before_points], ignore_index=True))

# before_d_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(uncluttered_data['lon'], uncluttered_data['lat']), crs=4326)
# before_d_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in uncluttered_data[['lon', 'lat']].itertuples(index=False, name=None)])], crs=4326)
# before_d_all = gpd.GeoDataFrame(pd.concat([before_d_linestring, before_d_points], ignore_index=True))

# after_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_mapmatchedGPS_points['LongitudeMatched'], df_mapmatchedGPS_points['LatitudeMatched']), crs=4326)
# after_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in df_mapmatchedGPS_points[['LongitudeMatched', 'LatitudeMatched']].itertuples(index=False, name=None)])], crs=4326)
# after_all = gpd.GeoDataFrame(pd.concat([after_linestring, after_points], ignore_index=True))

before = gpd.GeoDataFrame(geometry=[line], crs=4326)
after = gpd.GeoDataFrame(geometry=[sub_line], crs=4326)

#%% RAW & MAP-MATCHING ROUTES - DRAW MAP
m = folium.Map([51.49856419519174, -2.548053188420676], tiles='cartodbdark_matter', zoom_start=15)
# m = folium.Map([51.49856419519174, -2.548053188420676], tiles='OpenStreetMap', zoom_start=14)
# folium.GeoJson(before_all, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='rawGPS_points').add_to(m)
# folium.GeoJson(after_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='snapped_points').add_to(m)
# folium.GeoJson(estimated_route, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)
# folium.GeoJson(after_all, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)

folium.GeoJson(before, style_function=lambda x:{'color': 'blue', 'opacity': 0.5}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='blue', fill_opacity=0.5), name='rawGPS_points').add_to(m)
folium.GeoJson(point1_gpd, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='snapped_points').add_to(m)
folium.GeoJson(after, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='rawGPS_points_decluttered').add_to(m)
# folium.GeoJson(after_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='snapped_points').add_to(m)
# folium.GeoJson(after, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)


# folium.CircleMarker(location=[51.432333, -2.588721],
#                         radius=10,
#                         color='orange',
#                         weight=5).add_to(m)

# folium.CircleMarker(location=[51.468177, -2.592043],
#                         radius=10,
#                         color='yellow',
#                         weight=5).add_to(m)
# folium.CircleMarker(location=[51.50772108110192, -2.5623560818312554],
#                         radius=10,
#                         color='green',
#                         weight=5).add_to(m)


folium.LayerControl(position='topright', collapsed=False).add_to(m)
# m.save('mapmatching.html')
m

# Check by TripId

In [16]:
csv_file = '../data/processed/trips_matched/p_70_20240717.csv'
data = pd.read_csv(csv_file)
data['ValidUntilTime'] = pd.to_datetime(data['ValidUntilTime'])
data['RecordedAtTime'] = pd.to_datetime(data['RecordedAtTime'])
# data['EpochTime'] = data['RecordedAtTime'].apply(lambda x: x.timestamp())
data['Longitude'] = pd.to_numeric(data['Longitude'])
data['Latitude'] = pd.to_numeric(data['Latitude'])
# data['HexId'] = data.apply(lambda x: h3.geo_to_h3(x['Latitude'], x['Longitude'], resolution=hex_res),
#                                 axis=1) # Row wise operation
data.sort_values(by=['RecordedAtTime', 'ValidUntilTime', 'DatedVehicleJourneyRef', 'VehicleUniqueId', 'LineRef'], 
                    ascending=True, ignore_index=True, inplace=True)
data.drop_duplicates(subset=['RecordedAtTime', 'LineRef', 'DirectionRef', 'VehicleUniqueId', 'BlockRef', 'OriginAimedDepartureTime', 'DestinationAimedArrivalTime', 'DatedVehicleJourneyRef'], 
                        keep='first', inplace=True, ignore_index=True)

# sample_data = data.loc[
#     (data['LineRef'].astype(str)=='m1') &
#     (data['DirectionRef'].astype(str)=='inbound') &
#     # (str(data['OriginRef'])=='0100FBX18342') &
#     # (str(data['OriginAimedDepartureTime'])=='2024-07-14T20:55:00+00:00') &
#     # (str(data['BlockRef'])=='7001') &
#     (data['VehicleRef'].astype(str)=='FBRI-39335') &
#     (data['DatedVehicleJourneyRef'].astype(str)=='2300')   
# ]

sample_data = data.loc[
    (data['TripId'].astype(str)=='cbe76b10-3b87-ab12-3ee5-33b309345fa8')
].copy().reset_index(drop=True)

In [19]:
estimated_route = eval(sample_data['EstimatedRoute'][0])

before_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(sample_data['Longitude'], sample_data['Latitude']), crs=4326)
before_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in sample_data[['Longitude', 'Latitude']].itertuples(index=False, name=None)])], crs=4326)
before_all = gpd.GeoDataFrame(pd.concat([before_linestring, before_points], ignore_index=True))

# before_d_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(sample_data['lon'], sample_data['lat']), crs=4326)
# before_d_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in sample_data[['lon', 'lat']].itertuples(index=False, name=None)])], crs=4326)
# before_d_all = gpd.GeoDataFrame(pd.concat([before_d_linestring, before_d_points], ignore_index=True))

# after_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(sample_data['LongitudeMatched'], sample_data['LatitudeMatched']), crs=4326)
# after_linestring = gpd.GeoDataFrame(geometry=[LineString([x for x in sample_data[['LongitudeMatched', 'LatitudeMatched']].itertuples(index=False, name=None)])], crs=4326)
# after_all = gpd.GeoDataFrame(pd.concat([after_linestring, after_points], ignore_index=True))


#%% RAW & MAP-MATCHING ROUTES - DRAW MAP
m = folium.Map([51.49856419519174, -2.548053188420676], tiles='cartodbdark_matter', zoom_start=15)
# m = folium.Map([51.49856419519174, -2.548053188420676], tiles='OpenStreetMap', zoom_start=14)
# folium.GeoJson(before_all, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='rawGPS_points').add_to(m)
# folium.GeoJson(after_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='snapped_points').add_to(m)
# folium.GeoJson(estimated_route, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)
# folium.GeoJson(after_all, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)

folium.GeoJson(before_all, style_function=lambda x:{'color': 'blue', 'opacity': 0.5}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='blue', fill_opacity=0.5), name='rawGPS_points').add_to(m)
# folium.GeoJson(before_d_all, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='rawGPS_points_decluttered').add_to(m)
# folium.GeoJson(after_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='snapped_points').add_to(m)
folium.GeoJson(estimated_route, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='estimated_route').add_to(m)


# folium.CircleMarker(location=[51.45837, -2.59167],
#                         radius=10,
#                         color='yellow',
#                         weight=5).add_to(m)

# folium.CircleMarker(location=[51.458433, -2.591871],
#                         radius=10,
#                         color='yellow',
#                         weight=5).add_to(m)


folium.LayerControl(position='topright', collapsed=False).add_to(m)
# m.save('mapmatching.html')
m